<a href="https://colab.research.google.com/github/gvanhemert/DeepLearning/blob/MultiInputUpdate/deep_wave_mask_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# builtins
import locale
import math
import glob
import pathlib
import functools
import logging
import time
import datetime

# numerical stuff
#import pickle5 as pickle
import tables

import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, Lambda
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Input, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
#from tensorflow.python import ipu

#import libpvti as pvti

# plotting
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO)


In [3]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.94.32.114:8470']


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.94.32.114:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
data_path_train = 'gs://bathy_sample/processed/20211013/train_data_mask_no_schematic'
data_path_test = 'gs://bathy_sample/processed/20211013/test_data_mask_no_schematic'
#meta_data_path = '/mnt/poddata/data/bathy-emodnet-a-runs.h5'
#all_checkpoints_path = 'gs://bathy_sample/dnn/checkpoints'
all_checkpoints_path = '/content/drive/MyDrive/DeepLearning/Benchmark/checkpoints'
model_name = 'guus-2d-mlp-cnn-v1.0'
model_path = '/content/drive/MyDrive/DeepLearning/Benchmark/test_models'
checkpoints_path = all_checkpoints_path + '/' + model_name

learning_rate = 1e-3
n_epochs = 100
batch_size = 8 * tpu_strategy.num_replicas_in_sync
steps_per_execution = 1
steps_per_epoch = 112
validation_steps = 320
gradient_accumulation_steps_per_replica = 8
raster_shape = (256, 256, 1)

In [5]:
def tf_parse(eg):
    """parse an example (or batch of examples, not quite sure...)"""

    # here we re-specify our format
    # you can also infer the format from the data using tf.train.Example.FromString
    # but that did not work
    example = tf.io.parse_example(
        eg[tf.newaxis],
        {
            'height': tf.io.FixedLenFeature([], tf.int64),
            'width': tf.io.FixedLenFeature([], tf.int64),
            'depth': tf.io.FixedLenFeature([], tf.int64),
            'bathy': tf.io.FixedLenFeature([], tf.string),
            'hs': tf.io.FixedLenFeature([], tf.string),
            'tm01': tf.io.FixedLenFeature([], tf.string),
            'theta0x': tf.io.FixedLenFeature([], tf.string),
            'theta0y': tf.io.FixedLenFeature([], tf.string),
            'eta': tf.io.FixedLenFeature([], tf.float32),
            'zeta': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavex': tf.io.FixedLenFeature([], tf.float32),
            'theta_wavey': tf.io.FixedLenFeature([], tf.float32),
            'mask': tf.io.FixedLenFeature([], tf.string),
        },
    )
    bathy = tf.io.parse_tensor(example["bathy"][0], out_type="float32")
    bathy = tf.ensure_shape(bathy, raster_shape)    # ensure shape, to be able to train the model
    hs = tf.io.parse_tensor(example["hs"][0], out_type="float32")
    hs = tf.ensure_shape(hs, raster_shape)
    tm01 = tf.io.parse_tensor(example["tm01"][0], out_type="float32")
    theta0x = tf.io.parse_tensor(example["theta0x"][0], out_type="float32")
    theta0y = tf.io.parse_tensor(example["theta0y"][0], out_type="float32")
    eta = example["eta"]
    zeta = example["zeta"]
    theta_wavex = example["theta_wavex"]
    theta_wavey = example["theta_wavey"]
    mask = tf.io.parse_tensor(example["mask"][0], out_type="bool")
    #mask = tf.cast(mask, dtype="float32")
    mask = tf.ensure_shape(mask, raster_shape)
    #img_input = tf.concat([bathy,mask],-1)
    attr = tf.stack([eta, zeta, theta_wavex, theta_wavey], axis=1)
    attr = tf.reshape(attr,shape=[-1])
    output = (hs, tm01, theta0x, theta0y)
    output = hs
    return (bathy, attr), output

In [6]:
def get_files(data_path):
    files = tf.io.gfile.glob(data_path + "/" + "*.tfrecords")
    return files

def get_dataset(files):
    """return a tfrecord dataset with all tfrecord files"""
    dataset =  tf.data.TFRecordDataset(files)
    dataset = dataset.map(tf_parse)
    return dataset

In [7]:
def AttnBlock2D(x, g, inter_channel):

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1])(x)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1])(g)

    f = Activation('relu')(tf.keras.layers.add([theta_x, phi_g]))

    psi_f = Conv2D(1, [1, 1], strides=[1, 1])(f)

    rate = Activation('sigmoid')(psi_f)

    att_x = tf.keras.layers.multiply([x, rate])

    return att_x


def attention_up_and_concate(down_layer, layer):
    
    in_channel = down_layer.get_shape().as_list()[3]

    up = UpSampling2D(size=(2, 2))(down_layer)
    layer = AttnBlock2D(x=layer, g=up, inter_channel=in_channel // 4)
    
    my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
    
    concate = my_concat([up, layer])
    return concate

In [8]:
def full_model(cnn_input_shape, mlp_input_shape):#, mask_input_shape):
    
    mlp_input = Input(mlp_input_shape)
    cnn_input = Input(cnn_input_shape)
    #mask_input = Input(mask_input_shape)
    x = Dense(256, activation='relu')(mlp_input)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    mlp_output = Dense(4, activation='relu')(x)

    x = Conv2D(16, (3,3), padding="same")(cnn_input)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2D(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    #x = MaxPooling2D(pool_size=(2,2))(x1)

    x = Conv2D(32, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x1 = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x1)

    x = Conv2D(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    #x = MaxPooling2D(pool_size=(2,2))(x3)

    x = Conv2D(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x2 = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2))(x2)
    
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    cnn_output = BatchNormalization()(x)

    conv_shape = K.int_shape(cnn_output)

    #x = Flatten()(cnn_output)
    #x = Dense(32, activation="relu")(x)
    #x = BatchNormalization()(x)
    #x = Dense(32, activation="relu")(x)
    #x = BatchNormalization()(x)
    #x = Dense(32, activation="relu")(x)
    #x = BatchNormalization()(x)

    #x = Concatenate()([x,mlp_output])
    x = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3], activation="relu")(mlp_output)
    x = BatchNormalization()(x)

    x = Reshape((conv_shape[1],conv_shape[2],int(conv_shape[3])))(x)

    #Concatenate cnn_output before compression with MLP output
    x = tf.keras.layers.add([x, cnn_output])

    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(128, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    #x = attention_up_and_concate(x, x4)
    x = Conv2DTranspose(128, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2DTranspose(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    #x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(64, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = attention_up_and_concate(x, x2)
    x = Conv2DTranspose(32, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    #x = attention_up_and_concate(x, x2)
    x = Conv2DTranspose(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
        
    x = attention_up_and_concate(x, x1)
    x = Conv2DTranspose(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    
    x = Conv2DTranspose(16, (3,3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)

    x = Conv2DTranspose(1, (3,3), padding="same", activation="linear")(x)

    model = Model(inputs=[cnn_input, mlp_input], outputs = x)
    
    return model



In [9]:
train_files = get_files(data_path_train)
test_files = get_files(data_path_test)

train_dataset = get_dataset(train_files)
test_dataset = get_dataset(test_files)

train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

In [10]:
#for sample in train_dataset.take(1):
 #   print(repr(sample))

In [11]:
start = time.time()
print(time.ctime(start))
with tpu_strategy.scope():
    model = full_model((256, 256, 1), 4)
    opt = Adam(learning_rate=learning_rate, decay=learning_rate / n_epochs)
    model.compile(loss="mean_squared_error", optimizer=opt, steps_per_execution=steps_per_execution)
    
    callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        #monitor='val_mse',
        mode='max',
        save_best_only=True
    ),
    tf.keras.callbacks.CSVLogger(
        filename=checkpoints_path + '.csv')
    ]
    
    model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs)#, callbacks=callbacks)#, steps_per_epoch=steps_per_epoch)
    #model.save(model_path + '/' + model_name + '.h5')

end = time.time()
print(time.ctime(end))

Wed Nov 17 08:27:06 2021
Epoch 1/100
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


112/112 [==============================] - 526s 4s/step - loss: 41.8165 - val_loss: 31.1681
Epoch 2/100
112/112 [==============================] - 61s 540ms/step - loss: 12.8820 - val_loss: 14.4121
Epoch 3/100
112/112 [==============================] - 60s 535ms/step - loss: 4.5095 - val_loss: 10.4443
Epoch 4/100
112/112 [==============================] - 60s 535ms/step - loss: 4.1423 - val_loss: 7.1748
Epoch 5/100
112/112 [==============================] - 58s 517ms/step - loss: 3.8534 - val_loss: 4.2802
Epoch 6/100
112/112 [==============================] - 60s 536ms/step - loss: 3.7317 - val_loss: 2.4559
Epoch 7/100
112/112 [==============================] - 61s 540ms/step - loss: 3.5902 - val_loss: 1.8623
Epoch 8/100
112/112 [==============================] - 59s 524ms/step - loss: 3.4653 - val_loss: 1.9583
Epoch 9/100
112/112 [==============================] - 54s 481ms/step - loss: 3.3803 - val_loss: 2.0470
Epoch 10/100
112/112 [==============================] - 54s 477ms/step - 

KeyboardInterrupt: ignored

In [ ]:
print(start, end)
print("Starting time: ", time.ctime(start))
print("Ending time: ", time.ctime(end))
print("Time elapsed: ", datetime.timedelta(seconds=round(end - start)))
print('')
print("Number of parameters: ", model.count_params())
print("")
model.save(model_path + '/' + model_name + '.h5')

1636980355.6898663 1636986434.680863
Starting time:  Mon Nov 15 12:45:55 2021
Ending time:  Mon Nov 15 14:27:14 2021
Time elapsed:  1:41:19

Number of parameters:  83026599



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
logging.info("training model...")
# TODO properly compute steps for progress bar (low priority)
steps_per_epoch = len(train_files) * 10 // batch_size

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoints_path, 
        save_weights_only=True,
        monitor='val_mse',
        mode='max',
        save_best_only=True
    )
]
model.fit(x=train_dataset, validation_data=test_dataset, epochs=n_epochs, callbacks=callbacks)

In [ ]:
model.summary()
model.count_params()
#tf.keras.utils.plot_model(model, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 160         input_2[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          activation[0][0]                 
______________________________________________________________________________________________

8244633

In [ ]:
pd.read_hdf(meta_data_path)

,$\zeta$,$\theta_{wave}$,$\eta$,$bathy_i$,bathy_file,bathy,uuid,run_id,bathy_source
0,1.000000,0.000000,1.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",88c98d1f-6307-43b5-a8bd-0cc6b23e105e,a,emodnet
1,1.000000,0.000000,1.000000,462,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-13.052446, -12.792448, -12.792448, -12.5124...",b38a89ab-f021-42dc-808e-e116dd924a46,a,emodnet
2,1.000000,0.000000,50.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",318908b3-1ed1-4117-8226-6eaeb2846a24,a,emodnet
3,1.000000,0.000000,50.000000,462,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-13.052446, -12.792448, -12.792448, -12.5124...",dee4ae77-a3af-4e73-9f16-54b7ac589158,a,emodnet
4,1.000000,6.283185,1.000000,0,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.2872086, -1.2872086, -1.2872086, -1.48720...",e7bb6f7b-69db-4515-9dcf-6dc009d4d775,a,emodnet
...,...,...,...,...,...,...,...,...,...
1011,13.282544,3.631269,8.666409,100,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-21.735966, -21.735966, -21.635967, -21.6359...",65382ebd-1b80-4d75-9287-8bb7f2a41db0,a,emodnet
1012,13.581516,2.358778,29.419263,402,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[6.6918297, 6.6918297, 6.6918297, 6.6918297, ...",0402bcb5-7309-4662-8a18-93f1dccde9b4,a,emodnet
1013,8.416452,5.184157,14.352605,151,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[-1.3258033, -1.3258033, -1.3258033, -1.32580...",065553d1-f60d-4d7e-997e-0b07747682be,a,emodnet
1014,1.511433,5.741217,39.150158,367,/p/11207539-001-undeepwaves/emodnet/bathymetry...,"[[20.617117, 20.617117, 20.017117, 20.017117, ...",06577167-1089-49f4-aea7-2f736f9398ca,a,emodnet
